## 1.7 Solving the N-Puzzle Problem

The objective of this exercise is the application of search methods, with emphasis on informed
search methods and the A\* algorithm, to solve the well-known N-Puzzle problem. The desired
objective self for the puzzle is as follows (0 represents the empty space):

<table>
<tr><th>9Puzzle</th><th>16Puzzle</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 4   | 5   | 6   |
| 7   | 8   | 0   |


</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 1   | 2   | 3   | 4   |
| 5   | 6   | 7   | 8   |
| 9   | 10  | 11  | 12  |
| 13  | 14  | 15  | 0   |

</td>
</tr>
</table>

Starting from a given initial state, the goal is to determine which operations to perform to
solve the puzzle, reaching the desired objective self.


In [2]:
from copy import deepcopy

# definition of the problem
class NPuzzleState:

    def __init__(self, board, move_history=[]):
        # board(list[list[int]]) - the state of the board
        # move_history(list[list[list[int]]]) - the history of the moves up until this state
        self.board = deepcopy(board)
        (self.blank_row, self.blank_col) = self.find_blank()

        # create an empty array and append move_history
        self.move_history = [] + move_history + [self.board]

    def children(self):
        # returns the possible moves
        functions = [self.up, self.down, self.left, self.right]

        children = []
        for func in functions:
            child = func()
            if child:
                children.append(child)

        return children

    def find_blank(self):
        # finds the blank row and col
        for row in range(len(self.board)):
            for col in range(len(self.board[0])):
                if self.board[row][col] == 0:
                    return (row, col)

    def move(func):
        # decorator function to add to history everytime a move is made
        # functions with @move will apply this decorator
        def wrapper(self):
            state = NPuzzleState(self.board, self.move_history)
            value = func(state)
            if value:
                return state
            else:
                return None

        return wrapper

    @move
    def up(self):
        # moves the blank upwards
        if self.blank_row == 0:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row - 1][self.blank_col]
            self.board[self.blank_row - 1][self.blank_col] = 0
            self.blank_row -= 1
            return True

    @move
    def down(self):
        # moves the blank downwards
        if self.blank_row == len(self.board) - 1:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row + 1][self.blank_col]
            self.board[self.blank_row + 1][self.blank_col] = 0
            self.blank_row += 1
            return True

    @move
    def left(self):
        # moves the blank left
        if self.blank_col == 0:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row][self.blank_col - 1]
            self.board[self.blank_row][self.blank_col - 1] = 0
            self.blank_col -= 1
            return True

    @move
    def right(self):
        # moves the blank right
        if self.blank_col == len(self.board[0]) - 1:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row][self.blank_col + 1]
            self.board[self.blank_row][self.blank_col + 1] = 0
            self.blank_col += 1
            return True

    def is_complete(self):
        # checks if the board is complete
        for row in range(len(self.board)):
            for col in range(len(self.board[0])):
                if self.board[row][col] != row * len(self.board[0]) + col + 1 and self.board[row][col] != 0:
                    return False
        return True

    def __hash__(self):
        # to be able to use the state in a set
        return hash(str([item for sublist in self.board for item in sublist]))

    def __eq__(self, other):
        # compares the two matrices
        return [item for sublist in self.board for item in sublist] == [item for sublist in other.board for item in sublist]

def print_sequence(sequence):
    print("Steps:", len(sequence) - 1)
    # prints the sequence of states
    for state in sequence:
        for row in state:
            print(row)
        print()


def problems():
    return (
        NPuzzleState([[1, 2, 3], [5, 0, 6], [4, 7, 8]]),
        NPuzzleState([[1, 3, 6], [5, 2, 0], [4, 7, 8]]),
        NPuzzleState([[1, 6, 2], [5, 7, 3], [0, 4, 8]]),
        NPuzzleState([[5, 1, 3, 4], [2, 0, 7, 8], [
                     10, 6, 11, 12], [9, 13, 14, 15]]),
    )

**b)** Implement code to solve this problem using the “breadth-first” strategy (in this case
identical to "Uniform Cost"). 

In [3]:
def bfs(problem):
    # problem(NPuzzleState) - the initial state
    queue = [problem]
    visited = set() # to not visit the same state twice

    while queue:
        current = queue.pop()
        visited.add(current)

        if current.is_complete():
            return current.move_history
        
        for child in current.children():
            if child not in visited:
                queue.append(child)

    return None

# prints the sequence for the first problem using bfs
print_sequence(bfs(problems()[2]))

Steps: 2188
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 7, 3]
[4, 8, 0]

[1, 6, 2]
[5, 7, 0]
[4, 8, 3]

[1, 6, 2]
[5, 0, 7]
[4, 8, 3]

[1, 6, 2]
[0, 5, 7]
[4, 8, 3]

[1, 6, 2]
[4, 5, 7]
[0, 8, 3]

[1, 6, 2]
[4, 5, 7]
[8, 0, 3]

[1, 6, 2]
[4, 5, 7]
[8, 3, 0]

[1, 6, 2]
[4, 5, 0]
[8, 3, 7]

[1, 6, 2]
[4, 0, 5]
[8, 3, 7]

[1, 6, 2]
[0, 4, 5]
[8, 3, 7]

[1, 6, 2]
[8, 4, 5]
[0, 3, 7]

[1, 6, 2]
[8, 4, 5]
[3, 0, 7]

[1, 6, 2]
[8, 4, 5]
[3, 7, 0]

[1, 6, 2]
[8, 4, 0]
[3, 7, 5]

[1, 6, 2]
[8, 0, 4]
[3, 7, 5]

[1, 6, 2]
[0, 8, 4]
[3, 7, 5]

[1, 6, 2]
[3, 8, 4]
[0, 7, 5]

[1, 6, 2]
[3, 8, 4]
[7, 0, 5]

[1, 6, 2]
[3, 8, 4]
[7, 5, 0]

[1, 6, 2]
[3, 8, 0]
[7, 5, 4]

[1, 6, 2]
[3, 0, 8]
[7, 5, 4]

[1, 6, 2]
[0, 3, 8]
[7, 5, 4]

[1, 6, 2]
[7, 3, 8]
[0, 5, 4]

[1, 6, 2]
[7, 3, 8]
[5, 0, 4]

[1, 6, 2]
[7, 3, 8]
[5, 4, 0]

[1, 6, 2]
[7, 3, 0]
[5, 4, 8]

[1, 6, 2]
[7, 0, 3]
[5, 4, 8]

[1, 6, 2]
[0, 7, 3]
[5, 4, 8]

[0, 6, 2]
[1, 7, 3]
[5, 4, 8]

[6, 0, 2]
[1, 7, 3]
[5, 4, 

**c)** Implement code to solve this problem using Greedy Search and using the A*
Algorithm.

Suppose the following heuristics for these methods:
- H1 - Number of incorrect placed pieces;
- H2 - Sum of manhattan distances from incorrect placed pieces to their correct places. 

In [4]:
import heapq # we'll be using a heap to store the states

def greedy_search(problem, heuristic):
    # problem (NPuzzleState) - the initial state
    # heuristic (function) - the heuristic function that takes a board (matrix), and returns an integer
    setattr(NPuzzleState, "__lt__", lambda self, other: heuristic(self) < heuristic(other))
    states = [problem]
    visited = set() # to not visit the same state twice
    

    while states:
        current = heapq.heappop(states)
        visited.add(current)

        if current.is_complete():
            return current.move_history
        
        for child in current.children():
            if child not in visited:
                heapq.heappush(states, child)
        
    return None

def _preferential_position(number, side):
    # calculates the preferred position of a piece given its number
    # number (int) - the number of the piece
    # side (int) - the size of the side of the board (only for square boards)
    if number == 0:
        # if it is the last piece, it is 0
        row = col = side - 1
    else:
        # otherwise it is sequential, starting at 1
        row = number // side
        col = number % side - 1
    return (row, col)

def h1(state):
    # heuristic function 1
    # returns the number of incorrect placed pieces in the matrix
    board = state.board
    side = len(board) # the size of the side of the board (only for square boards)

    total = 0
    for row in range(side):
            for col in range(side):
                value = board[row][col]
                pos = _preferential_position(value,side)
                if (row,col) != pos:
                    total +=1

    return total

def h2(state):
    # heuristic function 2
    # returns the sum of manhattan distances from incorrect placed pieces to their correct places
    board = state.board
    side = len(board) # the size of the side of the board (only for square boards)

    total = 0
    for row in range(side):
            for col in range(side):
                value = board[row][col]
                pos = _preferential_position(value,side)
                total += abs(row-pos[0]) + abs(col-pos[1])

    return total

print('h1')
print_sequence(greedy_search(problems()[2], h1))

print('h2')
print_sequence(greedy_search(problems()[2], h2))


h1
Steps: 58
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 7, 3]
[4, 8, 0]

[1, 6, 2]
[5, 7, 0]
[4, 8, 3]

[1, 6, 2]
[5, 0, 7]
[4, 8, 3]

[1, 6, 2]
[0, 5, 7]
[4, 8, 3]

[1, 6, 2]
[4, 5, 7]
[0, 8, 3]

[1, 6, 2]
[4, 5, 7]
[8, 0, 3]

[1, 6, 2]
[4, 5, 7]
[8, 3, 0]

[1, 6, 2]
[4, 5, 0]
[8, 3, 7]

[1, 6, 2]
[4, 0, 5]
[8, 3, 7]

[1, 6, 2]
[4, 3, 5]
[8, 0, 7]

[1, 6, 2]
[4, 3, 5]
[0, 8, 7]

[1, 6, 2]
[0, 3, 5]
[4, 8, 7]

[1, 6, 2]
[3, 0, 5]
[4, 8, 7]

[1, 6, 2]
[3, 5, 0]
[4, 8, 7]

[1, 6, 0]
[3, 5, 2]
[4, 8, 7]

[1, 0, 6]
[3, 5, 2]
[4, 8, 7]

[1, 5, 6]
[3, 0, 2]
[4, 8, 7]

[1, 5, 6]
[3, 2, 0]
[4, 8, 7]

[1, 5, 0]
[3, 2, 6]
[4, 8, 7]

[1, 0, 5]
[3, 2, 6]
[4, 8, 7]

[1, 2, 5]
[3, 0, 6]
[4, 8, 7]

[1, 2, 5]
[0, 3, 6]
[4, 8, 7]

[1, 2, 5]
[4, 3, 6]
[0, 8, 7]

[1, 2, 5]
[4, 3, 6]
[8, 0, 7]

[1, 2, 5]
[4, 0, 6]
[8, 3, 7]

[1, 2, 5]
[4, 6, 0]
[8, 3, 7]

[1, 2, 5]
[4, 6, 7]
[8, 3, 0]

[1, 2, 5]
[4, 6, 7]
[8, 0, 3]

[1, 2, 5]
[4, 6, 7]
[0, 8, 3]

[1, 2, 5]
[0, 6, 7]
[4, 8,

In [5]:
def a_star_search(problem, heuristic):
    # problem (NPuzzleState) - the initial state
    # heuristic (function) - the heuristic function that takes a board (matrix), and returns an integer

    # this is very similar to greedy, the difference is that it takes into account the cost of the path so far
    return greedy_search(problem, lambda state: heuristic(state) + len(state.move_history)-1)


print('h1')
print_sequence(a_star_search(problems()[2], h1))

print('h2')
print_sequence(a_star_search(problems()[2], h2))


h1
Steps: 10
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 0, 3]
[4, 7, 8]

[1, 0, 2]
[5, 6, 3]
[4, 7, 8]

[1, 2, 0]
[5, 6, 3]
[4, 7, 8]

[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

h2
Steps: 10
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 0, 3]
[4, 7, 8]

[1, 0, 2]
[5, 6, 3]
[4, 7, 8]

[1, 2, 0]
[5, 6, 3]
[4, 7, 8]

[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



**d)** Compare the results obtained concerning execution time and memory space occupied
in solving the following problems using the previous methods

<table>
<tr><th>Prob. 1</th><th>Prob. 2</th><th>Prob. 3</th><th>Prob. 4</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 5   | 0   | 6   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 3   | 6   |
| 5   | 2   | 0   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 6   | 2   |
| 5   | 7   | 3   |
| 0   | 4   | 8   | 

</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 5   | 1   | 3   | 4   |
| 2   | 0   | 7   | 8   |
| 10  | 6   | 11  | 12  |
| 9   | 13  | 14  | 15  |

</td>
</tr>
</table>

In [6]:
import time

def test(title,fun,*args):
    start = time.time()
    res = fun(*args)
    end = time.time()
    print(f"{title}:\t{len(res)- 1} rounds\t {round(end-start,10)}s")

for i in range(4):
    print(f"prob{i+1}")

    #test('breadth-first',bfs,problems()[i])
    test('greedy h1',greedy_search,problems()[i],h1)
    test('greedy h2',greedy_search,problems()[i],h2)
    test('a-star h1',a_star_search,problems()[i],h1)
    test('a-star h2',a_star_search,problems()[i],h2)

    print()

prob1
greedy h1:	4 rounds	 0.0003318787s
greedy h2:	4 rounds	 0.0008304119s
a-star h1:	4 rounds	 0.0002834797s
a-star h2:	4 rounds	 0.0006451607s

prob2
greedy h1:	11 rounds	 0.0052015781s
greedy h2:	7 rounds	 0.0021483898s
a-star h1:	7 rounds	 0.0033700466s
a-star h2:	7 rounds	 0.0020225048s

prob3
greedy h1:	58 rounds	 0.0356023312s
greedy h2:	12 rounds	 0.0121901035s
a-star h1:	10 rounds	 0.0056622028s
a-star h2:	10 rounds	 0.0110914707s

prob4
greedy h1:	370 rounds	 1.7451162338s
greedy h2:	38 rounds	 0.0397074223s
a-star h1:	10 rounds	 0.0027999878s
a-star h2:	10 rounds	 0.0076746941s

